# ДЗ 3. Построение надежных схем валидации решения, оптимизация целевых метрик

**Основное задание:**  

Даны выборки для обучения и для тестирования. Задание заключается в том, чтобы попробовать разные способы валидации, проанализировать плюсы / минусы каждой и сделать выводы о том, какой способ валидации наиболее устойчивый в данной задаче. Метрика качества для оценки прогнозов - ROC-AUC, название целевой переменной - IsFraud. Рекомендуется использовать модели градиетного бустинга, реализация любая / гипепараметры любые.  

**Внимание!** выборка assignment_2_test.csv - наш аналог лидерборда. Будем моделировать ситуацию отправки решения на лидерборд и сравнить значение метрики на лидерборде и на локальной валидации.  Для других целей использовать выборку запрещено!  

**Терминалогия, используемая в задании:**  
* обучающая выборка - выборка, которая передается в метод fit / train;
* валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (train, valid);
* тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (train, valid, test);
* ЛБ - лидерборд, выборка assignment_2_test.csv.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import catboost as cb
from catboost import Pool
from catboost.utils import get_roc_curve

from tqdm import tqdm
from typing import List, Tuple

import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import ttest_rel

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GroupKFold
from sklearn.model_selection import train_test_split, TimeSeriesSplit

import warnings
warnings.simplefilter("ignore")

**Подготовим данные**

In [15]:
train = pd.read_csv('./data/assignment_2_train.csv')
board = pd.read_csv('./data/assignment_2_test.csv')

In [22]:
num_features = train.select_dtypes(exclude=[np.object]).columns.to_list()
num_train = train[num_features]
num_board = board[num_features]

In [23]:
X_data = num_train.drop('isFraud', axis=1)
y_data = num_train['isFraud']
X_board = num_board.drop('isFraud', axis=1)
y_board = num_board['isFraud']

## Hold-Out валидация с разбиением

**Задание 1:** сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [26]:
# Разобьем выборку
X_train, X_valid = train_test_split(X_data, train_size=0.7, shuffle=True, random_state=1)
y_train, y_valid = train_test_split(y_data, train_size=0.7, shuffle=True, random_state=1)

X_train.shape, y_train.shape,  X_valid.shape, y_valid.shape

((125999, 379), (125999,), (54001, 379), (54001,))

In [27]:
# Обучим модель
model = xgb.XGBClassifier(random_state=1)

model.fit(X=X_train, y=y_train, 
          eval_set=[(X_train, y_train), (X_valid, y_valid)], 
          early_stopping_rounds=20, 
          eval_metric="auc",
          verbose=20)

[0]	validation_0-auc:0.74944	validation_1-auc:0.74518
[20]	validation_0-auc:0.93829	validation_1-auc:0.90790
[40]	validation_0-auc:0.95999	validation_1-auc:0.92489
[60]	validation_0-auc:0.96989	validation_1-auc:0.93105
[80]	validation_0-auc:0.97807	validation_1-auc:0.93606
[99]	validation_0-auc:0.98292	validation_1-auc:0.93776


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [28]:
print(f"ROC AUC train = {round(roc_auc_score(y_train, model.predict_proba(X_train)[:, 1:]), 3)}")
print(f"ROC AUC valid = {round(roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1:]), 3)}")
print(f"ROC AUC board = {round(roc_auc_score(y_board, model.predict_proba(X_board)[:, 1:]), 3)}")

ROC AUC train = 0.982
ROC AUC valid = 0.938
ROC AUC board = 0.845


**Выводы:**  
* на обучающей и валидационной выборках наблюдается сильное переобучение
* на валидационной и лидерборд выборках наблюдается существенный разрыв метрик
* валидация с разбиением не является устойчивой

## Hold-Out валидация с разбиением на 3 выборки

**Задание 2:** сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [30]:
# Разобьем выборку на 3 выборки
X_train, X_valid = train_test_split(X_data, train_size=0.6, shuffle=True, random_state=1)
y_train, y_valid = train_test_split(y_data, train_size=0.6, shuffle=True, random_state=1)

X_valid, X_test = train_test_split(X_valid, train_size=0.5, shuffle=True, random_state=42)
y_valid, y_test = train_test_split(y_valid, train_size=0.5, shuffle=True, random_state=42)

X_train.shape, X_valid.shape, X_test.shape

((108000, 379), (36000, 379), (36000, 379))

In [31]:
# Обучим модель
model = xgb.XGBClassifier(random_state=1)

model.fit(X=X_train, y=y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
          early_stopping_rounds=20,
          eval_metric="auc",
          verbose=20)

[0]	validation_0-auc:0.74289	validation_1-auc:0.72581
[20]	validation_0-auc:0.94236	validation_1-auc:0.90305
[40]	validation_0-auc:0.96211	validation_1-auc:0.91841
[60]	validation_0-auc:0.97264	validation_1-auc:0.92293
[80]	validation_0-auc:0.97927	validation_1-auc:0.92805
[99]	validation_0-auc:0.98415	validation_1-auc:0.92902


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [32]:
print(f"ROC AUC train = {round(roc_auc_score(y_train, model.predict_proba(X_train)[:, 1:]), 3)}")
print(f"ROC AUC valid = {round(roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1:]), 3)}")
print(f"ROC AUC test  = {round(roc_auc_score(y_test, model.predict_proba(X_test)[:, 1:]), 3)}")
print(f"ROC AUC board = {round(roc_auc_score(y_board, model.predict_proba(X_board)[:, 1:]), 3)}")

ROC AUC train = 0.984
ROC AUC valid = 0.929
ROC AUC test  = 0.934
ROC AUC board = 0.838


**Выводы:** 
* по-прежнему на первых трех выборках наблючается сильное переобучение
* между выборкой лидерборда и другими по-прежнему существенный разрыв
* валидация с разбиением на 3 выборки так же не является устойчивой

## Доверительный интервал + качество модели

**Задание 3:** построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

**Подготовим функции**

In [34]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

**Оценим качество и построим доверительный интервал**

In [44]:
# Качество модели
np.random.seed(42)
scores = create_bootstrap_metrics(y_board, model.predict_proba(X_board)[:, 1:], roc_auc_score)
pd.DataFrame(scores).mean()

0    0.837744
dtype: float64

In [45]:
# Доверительный интервал
calculate_confidence_interval(scores)

(0.8301766452107935, 0.8454514275265516)

**Вывод:** метрика находится в пределах доверительного интервала, что говорит об устойчивости валидации на 3 выборках.

## Adversarial Validation + качество модели

**Задание 4:** выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

In [46]:
X_adv = pd.concat([X_data, X_board], axis=0)
y_adv = np.hstack((np.zeros(X_data.shape[0]), np.ones(X_board.shape[0])))

assert X_adv.shape[0] == y_adv.shape[0]

In [47]:
# Обучим модель
model = xgb.XGBClassifier(n_estimators=42)
model.fit(X_adv, y_adv)

[20:29:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=42, n_jobs=16, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [48]:
y_pred_adv = model.predict_proba(X_adv)
score = roc_auc_score(y_adv, y_pred_adv[:, 1])
print(round(score, 4))

1.0


In [49]:
y_pred = model.predict_proba(X_data)
y_pred

array([[9.9999416e-01, 5.8439687e-06],
       [9.9999416e-01, 5.8439687e-06],
       [9.9999416e-01, 5.8439687e-06],
       ...,
       [9.9999207e-01, 7.9173988e-06],
       [9.9999207e-01, 7.9173988e-06],
       [9.9999207e-01, 7.9173988e-06]], dtype=float32)

In [50]:
pd.cut(y_pred[:, 1], bins=np.arange(0, 1.01, 0.1)).value_counts().sort_index()

(0.0, 0.1]    180000
(0.1, 0.2]         0
(0.2, 0.3]         0
(0.3, 0.4]         0
(0.4, 0.5]         0
(0.5, 0.6]         0
(0.6, 0.7]         0
(0.7, 0.8]         0
(0.8, 0.9]         0
(0.9, 1.0]         0
dtype: int64

**Вывод:** объекты обучаемой выборки похожи на объекты валидационной выборки, значит их можно использовать для обучения модели.

## KFold/StratifiedKFold валидация

**Задание 5:** сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.

**Подготовим функции**

In [52]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy):
    """
    Кросс-валидация.

    Parameters
    ----------
    X: pd.DataFrame
        Матрица признаков.

    y: pd.Series
        Вектор целевой переменной.

    estimator: callable
        Объект модели для обучения.

    metric: callable
        Метрика для оценки качества решения.
        Ожидается, что на вход будет передана функция,
        которая принимает 2 аргумента: y_true, y_pred.

    cv_strategy: cross-validation generator
        Объект для описания стратегии кросс-валидации.
        Ожидается, что на вход будет передан объект типа
        KFold или StratifiedKFold.

    Returns
    -------
    oof_score: float
        Значение метрики качества на OOF-прогнозах.

    fold_train_scores: List[float]
        Значение метрики качества на каждом обучающем датасете кросс-валидации.

    fold_valid_scores: List[float]
        Значение метрики качества на каждом валидационном датасете кросс-валидации.

    oof_predictions: np.array
        Прогнозы на OOF.

    """
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        estimator.fit(x_train, y_train)
        y_train_pred = estimator.predict(x_train)
        y_valid_pred = estimator.predict(x_valid)

        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [55]:
cv_strategy = KFold(n_splits=5)
estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(X_data, y_data, 
                                                                                                     model, 
                                                                                                     metric=roc_auc_score, 
                                                                                                     cv_strategy=cv_strategy)

[20:34:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.7744, valid-score = 0.6374
[20:34:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold: 2, train-observations = 144000, valid-observations = 36000
train-score = 0.7749, valid-score = 0.6651
[20:34:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used 

In [56]:
cv_strategy = KFold(n_splits=5)
estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(X_board, y_board, 
                                                                                                     model, 
                                                                                                     metric=roc_auc_score, 
                                                                                                     cv_strategy=cv_strategy)

[20:36:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold: 1, train-observations = 80000, valid-observations = 20001
train-score = 0.777, valid-score = 0.67
[20:36:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold: 2, train-observations = 80001, valid-observations = 20000
train-score = 0.7827, valid-score = 0.6401
[20:36:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with 

**Вывод:** наблюдаем не очень большую дисперсии на основании результатов обучающей и лидерборд выборок, можем утверждать, что KFold валидация является достаточно устойчивой.

## Hold-Out валидацию по времени

**Задание 6 (опциональное):** сделать Hold-Out валидацию по времени (TransactionDT), повторить процедуры из п.1 / п.2 (на ваш выбор). Построить доверительный интервал, сравнить качество на ЛБ выборке с полученным доверительным интервалом. Сделать выводы.

In [57]:
cv = TimeSeriesSplit()

for train_index, test_index in cv.split(X_data):
    X_train, X_valid = X_data.loc[train_index], X_data.loc[test_index]
    y_train, y_valid = y_data[train_index], y_data[test_index]
    
X_train.shape,  X_valid.shape, y_train.shape, y_valid.shape

((150000, 379), (30000, 379), (150000,), (30000,))

In [58]:
model = xgb.XGBClassifier(random_state=1)

model.fit(X=X_train, y=y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
          early_stopping_rounds=20,
          eval_metric="auc",
          verbose=20)

[0]	validation_0-auc:0.76162	validation_1-auc:0.75636
[20]	validation_0-auc:0.93785	validation_1-auc:0.89197
[40]	validation_0-auc:0.95839	validation_1-auc:0.90488
[60]	validation_0-auc:0.96540	validation_1-auc:0.90367
[62]	validation_0-auc:0.96578	validation_1-auc:0.90387


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [59]:
print(f"ROC AUC train = {round(roc_auc_score(y_train, model.predict_proba(X_train)[:, 1:]), 3)}")
print(f"ROC AUC valid = {round(roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1:]), 3)}")
print(f"ROC AUC board = {round(roc_auc_score(y_board, model.predict_proba(X_board)[:, 1:]), 3)}")

ROC AUC train = 0.959
ROC AUC valid = 0.906
ROC AUC board = 0.853


In [60]:
# Качество модели
np.random.seed(42)
scores = create_bootstrap_metrics(y_board, model.predict_proba(X_board)[:, 1:], roc_auc_score)
pd.DataFrame(scores).mean()

0    0.853181
dtype: float64

In [61]:
# Доверительный интервал
calculate_confidence_interval(scores)

(0.8462826718303791, 0.8603543072426955)

**Вывод:** метрика выборки лидерборда входит в доверительный интервал, но ее нельзя назвать устойчивой, т.к. наблюдается значимый разрыв между метриками обучающей и лидерборд выборок.

## GroupKFold валидация

**Задание 7 (совсем опциональное):** в данном наборе данных у нас есть ID-транзакции (TransactionID) и время транзакции (TransactionDT), но отсутствует ID-клиента, который совершал транзакции. Кажется, что в этой задаче валидация по клиенту работала бы хорошо. Предложить критерий, по которому можно выделить клиентов и сделать п.5, используя созданное определение клиента, используя валидацию по клиенту (GroupKFold).

**Подготовим функции**

In [63]:
def get_clientID(data):
    
    for n, k in enumerate(data['ProductCD'].value_counts().keys()):
        data.loc[data['ProductCD'] == k, 'ProdCD'] = n
    data['ProdCD'] = data['ProdCD'].astype(np.int8)

    data.loc[data['P_emaildomain'].isnull(), 'P_emaildomain'] = 'nan'
    for n, k in enumerate(data['P_emaildomain'].value_counts().keys()):
        data.loc[data['P_emaildomain'] == k, 'P_email'] = n
    data['P_email'] = data['P_email'].astype(np.int8)

    data['ClientID'] = data['ProdCD'].astype(str) + data['card1'].astype(str) + data['P_email'].astype(str)
    data['ClientID'] = data['ClientID'].astype(np.int32)

    data = data.drop(['ProdCD', 'P_email'], axis=1)
    
    num_features = data.select_dtypes(exclude=[np.object]).columns.to_list()
    data_num = data[num_features]
    
    X_data = data_num.drop('isFraud', axis=1)
    y_data = data_num['isFraud']
    
    return X_data, y_data

In [65]:
X_data, y_data = get_clientID(train)
X_board, y_board = get_clientID(board)

In [66]:
def make_cross_validation_groups(X: pd.DataFrame,
                              y: pd.Series,
                              estimator: object,
                              metric: callable,
                              cv_strategy,
                              groups):

    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y, groups=X[groups])):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        estimator.fit(x_train, y_train)
        y_train_pred = estimator.predict(x_train)
        y_valid_pred = estimator.predict(x_valid)

        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [67]:
cv_strategy = GroupKFold(n_splits=5)
estimators, oof_score, fold_train_scores, 
fold_valid_scores, oof_predictions = make_cross_validation_groups(X_data, y_data,
                                                                  model, 
                                                                  metric=roc_auc_score, 
                                                                  cv_strategy=cv_strategy, 
                                                                  groups='ClientID')

[20:47:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.7775, valid-score = 0.6726
[20:48:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold: 2, train-observations = 144000, valid-observations = 36000
train-score = 0.7696, valid-score = 0.6511
[20:48:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used 

In [69]:
cv_strategy = GroupKFold(n_splits=5)
estimators, oof_score, fold_train_scores, 
fold_valid_scores, oof_predictions = make_cross_validation_groups(X_board, y_board, 
                                                                  model, 
                                                                  metric=roc_auc_score, 
                                                                  cv_strategy=cv_strategy, 
                                                                  groups='ClientID')

[20:51:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold: 1, train-observations = 80000, valid-observations = 20001
train-score = 0.7816, valid-score = 0.6911
[20:51:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold: 2, train-observations = 80001, valid-observations = 20000
train-score = 0.7808, valid-score = 0.6365
[20:51:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used wi

ValueError: too many values to unpack (expected 2)

**Вывод:** судя по метрике валидация GroupKFold по предположительному клиенту является стабильной.